Import patra trabajar con GCP

In [ ]:
pip install apache-beam[gcp]

#After install apache beam is important reset the enviorment

In [ ]:
pip install --upgrade firebase-admin

# Configuramos todo

In [4]:
#BIGQUERY IMPORTS
from google.cloud import bigquery #For bigquerycredentials
import apache_beam as beam #For apacheBeam
from apache_beam.options.pipeline_options import PipelineOptions #For beam.io.ReadFromBigQuery

##FIRESTORE imports
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json #Used to format object before save in firestore

#FIRESTORE AND BIGQUERY IMPORTS
from google.oauth2 import service_account #For credentials


##Mount access to Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

#Set credentials as global
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="drive/MyDrive/Ideas/movilidad/vacio-276411-service_account_for_colab.json"

##Get credentials
key_path = 'drive/MyDrive/Ideas/movilidad/vacio-276411-service_account_for_colab.json'
credentialsBigQuery = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentialsBigQuery)

#Options for ReadFromBigQuery
options = PipelineOptions(
    temp_location='gs://colab_temp/temp',
    project ='vacio-276411')


#FIRESTORE configuration
credentials_firestore = credentials.Certificate(key_path)

#firebase_admin.initialize_app(credentials_firestore, name="writeToFirestore17")
firebase_admin.initialize_app(credentials_firestore)
db_firestore = firestore.client()

Mounted at /content/drive


Constants

In [61]:
#CONSTANTS
table_spec_bulkData_b = 'vacio-276411.mainDataset.bulkData_b'
firestore_actual_info = "vehicles_current_b"
#DIC global for all notebook
dict_old_locations = {}
last_uoid = ''

Variable para debuguear procesos

In [7]:
in_debug = False

# Recogemos datos de Firestore

In [62]:
def get_firestore_old_situation():
  #CONSTANTS

  docs = db_firestore.collection(firestore_actual_info).get()

  #TODO: FInd another idea to convert Firebase collection to python dict
  for doc in docs:
    #print(f'{doc.id} => {doc.to_dict()}')
    dict_old_locations[doc.id] = doc.to_dict()

  print('Vehicles in Firestore: ' +  str(len(dict_old_locations)))
  #print(dict_old_locations)



# Comparamos las ubicaciones antiguas con las nuevas

Recogemos el id de los datos mas recientes

In [84]:
def get_last_uoid():
  query = """
      SELECT uoid, epochTime FROM `vacio-276411.mainDataset.bulkData_b` 
      ORDER BY epochTime DESC
      LIMIT 1
  """
  query_job = client.query(query)  # Make an API request.


  print("The query data:") if in_debug else ''
  for row in query_job:
      print("name={}".format(row[0])) if in_debug else ''
      last_uoid = row[0]

  return last_uoid

Cantidad de vehiculos en este flujo (no necesario para el proceso)

In [64]:
query = ' '.join(("SELECT COUNT(*) FROM `vacio-276411.mainDataset.bulkData_b`",
        " WHERE uoid = '" + last_uoid +"'"))

query_job = client.query(query)  # Make an API request.

for row in query_job:
    print("In this iteration there are: " + str(row[0]) + " vehicles")

In this iteration there are: 156 vehicles


### Funcion para guardar en Firestore
- Necesita convertir el formato de los datos BigQuery NUMERIC a float para poder guardarlos en Firestore NUMERIC

In [65]:
def prepare_to_save_in_firestore(row):
  print('[FUNC]prepare_to_save_in_firestore') if in_debug else ''
  row['energia'] = float(row['energia'])
  row['latitud'] = float(row['latitud'])
  row['longitud'] = float(row['longitud'])
  row['epochTime'] = float(row['epochTime'])
  row['autonomyValue'] = float(row['autonomyValue'])
  row['seats'] = float(row['seats'])
  print(row) if in_debug else ''
  return row

### TODO: Add comment

In [74]:
def save_firestore(row):
    print('[FUNC]save_firestore') if in_debug else ''
    print(row) if in_debug else ''
    key = row['matricula']
    print(key) if in_debug else ''
    doc_ref = db_firestore.collection(firestore_actual_info).document(key)
    doc_ref.set(row)
    #Return row to continue with data
    return row

### TODO: Add comment

In [71]:
def check_movements(row):
  print('[FUNC]check_movements') if in_debug else ''
  plate = row['matricula']

  #Check if plate exist in firebase database
  if plate not in dict_old_locations.keys():
    print('[check_movements]New vehicle')
    print(row)
    #NO process
    return row

  new_latitude = row['latitud']
  old_latitude = dict_old_locations[plate]['latitud']

  new_longitude = row['longitud']
  old_longitude = dict_old_locations[plate]['longitud']

  if(new_latitude == old_latitude and new_longitude == old_longitude):
    print('[check_movements] Son iguales') if in_debug else ''
  else:
    print('[check_movements] Son distintas')
    print(new_latitude, old_latitude, new_longitude, old_longitude)

  return row

### Get new information from BigQuery

In [90]:
""" QUERY ONLY TWO VEHICLES
query = ' '.join(("SELECT * FROM `vacio-276411.mainDataset.bulkData_b`"
        " WHERE idVehiculo = 'ZACNJDAB0MPN19718' OR idVehiculo = '1G1BE5SM0K7113537' ",
        "ORDER BY realTime DESC",
        "LIMIT 2"))
"""

def update_query(last_uoid):
  query = ' '.join(("SELECT * FROM `vacio-276411.mainDataset.bulkData_b`"
          " WHERE uoid = '" + last_uoid +"'",
          "ORDER BY realTime DESC"))
  return query


### Print row

In [69]:
def print_row(row):
    print(row)

In [95]:
from datetime import datetime
def print_time(row):
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  print("Current Time =", current_time)
  return row

### TODO: Add comment

In [111]:
#First update old situation
get_firestore_old_situation()
#Get last uoid
last_uoid =  get_last_uoid()
#Update query with last uoid
last_uoid = '27f342be-174a-4278-88aa-8fbb8b9ce0e6'
last_uoid = 'a2e360d5-e5b9-4cac-8893-ac9d911a94f3'
last_uoid = '9719a677-68ee-46b8-befd-1b15c62c5012'
query = update_query(last_uoid)


pipeline = beam.Pipeline(options=options)
vehicles_location = (
    pipeline
    | 'QueryTable' >> beam.io.ReadFromBigQuery(query=query, use_standard_sql=True)
    #| 'PrintTime1' >> beam.Map(print_time)
    | "FormatToFirestore" >> beam.Map(prepare_to_save_in_firestore)
    | "SaveInFirestore" >> beam.Map(save_firestore)
    | "CalculateMovements" >> beam.Map(check_movements)
    #| "print" >> beam.Map(print_row)
)

#pipeline.run()

result = pipeline.run()
metrics = result.metrics()
metricResults = metrics.GAUGES
metricResults


Vehicles in Firestore: 619


[check_movements] Son distintas
40.464288 40.454182 -3.688986 -3.682985
[check_movements] Son distintas
40.458051 40.472157 -3.697244 -3.68371
[check_movements] Son distintas
40.461283 40.456496 -3.674118 -3.679822
[check_movements] Son distintas
40.489222 40.402858 -3.64807 -3.677254
[check_movements] Son distintas
40.438182 40.477366 -3.684801 -3.699015
[check_movements] Son distintas
40.449798 40.444026 -3.687211 -3.665838
[check_movements] Son distintas
40.42729 40.476864 -3.701186 -3.738568
[check_movements]New vehicle
{'city': 'Madrid', 'servicio': 'free2move', 'idVehiculo': 'VR3UHZKXZMT016708', 'matricula': '3408LNB', 'energia': 39.0, 'latitud': 40.418336, 'longitud': -3.664079, 'tipo': 'coche', 'categoria': 'Peugeot e-208', 'imagen': 'e-208-white.png', 'uoid': '9719a677-68ee-46b8-befd-1b15c62c5012', 'epochTime': 1659600000.3161585, 'realTime': '08/04/2022 08:00:00', 'geo': 'POINT(-3.664079 40.418336)', 'timestamp': datetime.datetime(2022, 8, 4, 6, 0, 0, 316159, tzinfo=datetime.

'gauges'